In [1]:
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)

if not dir1 in sys.path:
    sys.path.append(dir1)

os.chdir('..')

In [2]:
import datetime
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
import yaml


from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import mean_absolute_percentage_error as MAPE
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

from src.data.preprocessing import read_data, data_to_np_tensor, preprocess_split
from src.data.load_data import pipeline_data
from src.models.ts2vec_src.ts2vec import TS2Vec

from src.experiments import LagModelExperint, SelfSupervisedExperint

%load_ext autoreload
%autoreload 2

# Config

In [3]:
col_agg_finctions = {'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}

In [4]:
train_start, train_end = '2023-10-01', '2023-11-01'
test_start, test_end = '2023-11-01', '2023-11-07'

In [5]:
use_pct_changes_X = False
use_pct_changes_ts2v = False
use_pct_changes_labels = False

In [6]:
ts2vec_device = 1
ts2vec_out_dim = 128

n_shifts = 18

In [7]:
ticker_data_path = 'configs/best_stocks_nans_rate.yaml'
ticker_data_path = 'data/all_tickers.csv'

# DataLoading

In [8]:
df = pipeline_data(col_agg_finctions=col_agg_finctions)

In [9]:
emb_model = TS2Vec(input_dims=4, device=2, output_dims=ts2vec_out_dim)
lag_model_exp = SelfSupervisedExperint(
    model = LinearRegression(),
    emb_model = emb_model,
    train_start = train_start, 
    train_end = train_end, 
    test_start = test_start, 
    test_end = test_end,
    label_name = 'Close',
    use_pct_changes_data = False,
    use_pct_changes_labels = False,
)

results, preds = lag_model_exp.pipeline(df)
results

NAN in data  0
NAN in data  0
NAN in data  0
NAN in data  0
NAN in data  0
NAN in data  0
NAN in data  0
NAN in data  0


/home/stock_embeddings/miniconda/lib/python3.9/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
/home/stock_embeddings/miniconda/lib/python3.9/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


0.8910746680924243

In [10]:
lag_model_exp = LagModelExperint(
    lag_model = LinearRegression(),
    train_start = train_start, 
    train_end = train_end, 
    test_start = test_start, 
    test_end = test_end,
    label_name = 'Close',
    use_pct_changes_data = False,
    use_pct_changes_labels = True,
)

results, preds = lag_model_exp.pipeline(df)
results

ValueError: Length mismatch: Expected axis has 3 elements, new values have 1 elements